# 실습 세션 3: 여행 플래너 Multi-Agent 앱의 Docker 빌드 및 AKS 배포

이 세션에서는 Docker로 앱을 컨테이너화하고, AKS에 배포하여 실제 클라우드 환경에서 운영하는 전체 과정을 단계별로 실습합니다.

# Docker, Kubernetes, AKS 한눈에 보기

## Docker
- **개념**: 앱과 실행 환경을 하나로 묶어 어디서나 똑같이 실행하는 컨테이너 기술
- **역사**: 2013년 등장, 컨테이너 대중화의 시작
- **이점**: 환경 일관성, 빠른 배포, 가벼움, 이식성

## Kubernetes
- **개념**: 컨테이너 수천 개를 자동으로 관리하는 오케스트레이션 플랫폼
- **역사**: 2014년 Google이 오픈소스로 공개, 업계 표준으로 자리잡음
- **이점**: 자동 배포/확장/복구, 멀티 클라우드 지원, 고가용성

## AKS
- **개념**: Azure에서 제공하는 완전관리형 Kubernetes 서비스
- **역사**: 2017년 출시, Kubernetes 운영을 Azure가 자동화
- **이점**: 인프라 관리 부담↓, Azure 서비스와 통합, 손쉬운 확장

## Docker로 할 수 있는 주요 작업 (주요 명령어 예시)
- **이미지 빌드**: `docker build -t <이미지명>:<태그> .`
- **컨테이너 실행**: `docker run -d --name <컨테이너명> -p <호스트포트>:<컨테이너포트> <이미지명>:<태그>`
- **실행 중인 컨테이너 목록 확인**: `docker ps`
- **전체 컨테이너 목록 확인**: `docker ps -a`
- **이미지 목록 확인**: `docker images`
- **컨테이너 중지/시작/재시작**: `docker stop <컨테이너명>`, `docker start <컨테이너명>`, `docker restart <컨테이너명>`
- **컨테이너 삭제**: `docker rm <컨테이너명>`
- **이미지 삭제**: `docker rmi <이미지명>:<태그>`
- **컨테이너 로그 확인**: `docker logs <컨테이너명>`
- **컨테이너 내부 접속**: `docker exec -it <컨테이너명> /bin/bash`
- **이미지 푸시/풀**: `docker push <이미지명>:<태그>`, `docker pull <이미지명>:<태그>`

## Kubernetes & AKS 아키텍처

```
+-------------------+         +-------------------+
|   Control Plane   | <-----  |   Azure Managed   |
| (API Server, etc) |         |   (AKS 관리 영역) |
| - API Server      |
| - Scheduler       |
| - Controller Mgr  |
| - etcd DB         |
| - Cloud Controller|
+-------------------+         +-------------------+
           |
           v
+-------------------+    +-------------------+    +-------------------+
|      Node 1       |    |      Node 2       |    |      Node N       |
|  (VM, Worker)     |    |  (VM, Worker)     |    |  (VM, Worker)     |
| - kubelet         |    | - kubelet         |    | - kubelet         |
| - kube-proxy      |    | - kube-proxy      |    | - kube-proxy      |
| - Container Runtime|   | - Container Runtime|   | - Container Runtime|
| - (옵션: CSI, CNI)|   | - (옵션: CSI, CNI)|   | - (옵션: CSI, CNI)|
+-------------------+    +-------------------+    +-------------------+
      |       |                |       |                |       |
      v       v                v       v                v       v
   +-----+ +-----+          +-----+ +-----+          +-----+ +-----+
   |Pod  | |Pod  |   ...    |Pod  | |Pod  |   ...    |Pod  | |Pod  |
   +-----+ +-----+          +-----+ +-----+          +-----+ +-----+
      |       |                |       |                |       |
      v       v                v       v                v       v
   [Container][Container]   [Container][Container]   [Container][Container]
```

### Control Plane (제어 플레인, 마스터 노드)
- **API Server**: 클러스터의 모든 명령과 요청을 받아들이는 중앙 진입점. kubectl, 클라우드 API 등과 통신.
- **Scheduler**: 새로 생성된 파드를 어떤 노드에 배치할지 결정.
- **Controller Manager**: 파드, 노드, 레플리카 등 클러스터 상태를 지속적으로 감시·조정.
- **etcd**: 클러스터의 모든 상태와 설정 정보를 저장하는 분산 Key-Value 저장소.
- **Cloud Controller Manager**: 클라우드 리소스(로드밸런서, 볼륨 등)와 Kubernetes를 연동.
- **(AKS 관리 영역)**: Control Plane 전체를 Azure가 완전 관리(보안, 업그레이드, 장애 복구 등).

### Data Plane (데이터 플레인, 워커 노드)
- **Node(노드)**: 실제 컨테이너가 실행되는 VM. 여러 노드로 클러스터 구성.
- **kubelet**: 각 노드에서 파드의 생성·삭제·상태를 관리하는 에이전트.
- **kube-proxy**: 네트워크 프록시 및 로드밸런서 역할. 파드 간/외부 통신을 지원.
- **Container Runtime**: 컨테이너 실행 환경(Docker, containerd 등).
- **CSI(CSI 플러그인)**: 외부 스토리지 연동(옵션).
- **CNI(CNI 플러그인)**: 네트워크 플러그인(옵션).
- **Pod**: 하나 이상의 컨테이너가 묶여 배포되는 최소 단위.
- **Container**: 실제 애플리케이션이 동작하는 환경.
- **Deployment**: 파드의 선언적 배포, 업데이트, 롤백, 확장 등을 자동으로 관리하는 리소스.
- **ReplicaSet**: 지정한 파드의 복제본(Replica) 수를 보장하며, 파드의 생성/삭제를 자동으로 조정.
- **Service**: 파드 집합에 고정된 네트워크 엔드포인트를 제공, 내부/외부 접근을 지원.
- **Ingress**: 외부 트래픽을 내부 서비스로 라우팅하는 진입점 역할, 도메인/경로 기반 라우팅 지원.

# AKS 글로벌 고객사례

- **Forza Horizon 5 (Xbox Game Studios)**: 전 세계 수백만 동시 접속을 지원하는 인기 레이싱 게임. AKS의 글로벌 분산, 자동 확장, 실시간 모니터링으로 대규모 트래픽에도 끊김 없는 게임 경험 제공.

- **BMW & MyBMW**: BMW는 제조·물류·차량 데이터 분석뿐 아니라, MyBMW 앱(글로벌 커넥티드카 서비스)도 AKS 기반으로 운영. 수백만 차량의 실시간 원격 제어, OTA 업데이트 등 혁신적 커넥티드 서비스를 안정적으로 제공.

- **Coca-Cola**: 글로벌 음료 제조·유통 시스템, 주문·재고 관리, 실시간 데이터 분석을 AKS로 운영. 대규모 트래픽과 글로벌 서비스 확장에 신속하게 대응.

- **Bosch**: IoT 디바이스 관리, 제조 데이터 분석, AI 기반 예지정비 등 다양한 산업 솔루션을 AKS에서 운영. 글로벌 공장 및 현장에 신속한 서비스 배포 실현.

- **Shell**: 에너지·석유화학 분야의 데이터 분석, IoT 센서 데이터 수집, AI 기반 운영 최적화 시스템을 AKS로 구축. 대규모 데이터 파이프라인과 글로벌 서비스 확장에 성공.

---

## 1. Docker CLI 설치 확인 (GitHub Codespaces 환경)

GitHub Codespaces 환경에는 이미 Docker CLI가 설치되어 있으므로 별도의 설치가 필요하지 않습니다.

- Codespaces 터미널에서 아래 명령어로 Docker CLI가 정상적으로 설치되어 있는지 확인하세요.

```sh
docker --version
```

- 위 명령어 실행 시 Docker 버전 정보가 출력되면 정상입니다.
- 로컬 환경에서 실습할 경우 [Docker Desktop](https://www.docker.com/products/docker-desktop/)을 별도로 설치해야 합니다.

## 2. Dockerfile 작성

아래는 여행 플래너 앱을 컨테이너화하기 위한 Dockerfile 예시입니다.

- Dockerfile은 컨테이너의 실행 환경, 의존성, 복사할 파일, 실행 명령어 등을 정의하는 핵심 파일입니다.
- 실습에서는 Python 기반 앱을 위한 베이스 이미지, 의존성 설치, 소스 복사, 엔트리포인트 지정 등 기본적인 Dockerfile 구조를 익힐 수 있습니다.
- Dockerfile을 수정하면 다양한 환경(예: 추가 패키지, 환경 변수, 빌드 단계 등)에 맞게 이미지를 커스터마이즈할 수 있습니다.
- 작성한 Dockerfile은 이후 단계에서 이미지 빌드 및 배포에 활용됩니다.

### 주요 Dockerfile 키워드 설명
- **FROM**: 베이스 이미지를 지정합니다. (예: python:3.12-slim)
- **WORKDIR**: 컨테이너 내 작업 디렉터리를 설정합니다.
- **COPY**: 파일 또는 디렉터리를 컨테이너 이미지로 복사합니다.
- **RUN**: 컨테이너 빌드 시 실행할 명령어를 지정합니다. (예: 패키지 설치)
- **ENTRYPOINT**: 컨테이너가 시작될 때 실행할 기본 명령어를 지정합니다.

아래 셀의 예시 Dockerfile을 참고하여, 필요에 따라 키워드와 명령어를 추가·수정해보세요.

In [1]:
%%writefile Dockerfile
# Step 1: Base image with Python environment
FROM python:3.12-slim AS base

# Step 2: Set working directory
WORKDIR /app

# Step 3: Install dependencies
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

# Step 4: Copy the project files into the container
COPY . .

# Step 5: Define the entry point
ENTRYPOINT ["python", "/app/travel_planning_agent.py"]

Writing Dockerfile


## 3. Docker 이미지 빌드

이 단계에서는 작성한 Dockerfile을 기반으로 실제 컨테이너 이미지를 빌드합니다.

- `docker build` 명령어를 사용해 소스코드와 Dockerfile이 위치한 디렉터리에서 이미지를 생성합니다.
- 빌드된 이미지는 로컬 환경에서 테스트하거나, 이후 Azure Container Registry(ACR)로 푸시할 수 있습니다.
- 이미지 태그(`-t` 옵션)를 활용해 버전 관리 및 ACR 업로드용 이름을 지정할 수 있습니다.
- 빌드 과정에서 오류가 발생하면 Dockerfile의 경로, 의존성, COPY/RUN 명령어 등을 점검하세요.
- 빌드가 완료되면 `docker images` 명령어로 생성된 이미지를 확인할 수 있습니다.

In [2]:
# Docker 이미지 빌드
!docker build -t travel-planning-agent-app:v1 .

[+] Building 0.0s (0/0)  docker:desktop-linux
[+] Building 0.0s (0/1)                                    docker:desktop-linux
[+] Building 0.0s (0/1)                                    docker:desktop-linux
[+] Building 0.2s (1/2)                                    docker:desktop-linux
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 455B                                       0.0s
 => [internal] load metadata for docker.io/library/python:3.12-slim        0.2s
[+] Building 0.2s (1/2)                                    docker:desktop-linux
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 455B                                       0.0s
 => [internal] load metadata for docker.io/library/python:3.12-slim        0.2s
[+] Building 0.3s (1/2)                                    docker:desktop-linux
 => [internal] load build definition from Dockerfile                      

## 4. Azure CLI 설치 (GitHub Codespaces 환경)

이 단계에서는 Azure 리소스(AKS, ACR 등)를 생성하고 관리하기 위해 필요한 Azure CLI(az)를 설치합니다.

- Azure CLI는 Azure 리소스의 생성, 관리, 배포, 모니터링 등 다양한 작업을 명령줄에서 수행할 수 있는 도구입니다.
- GitHub Codespaces 환경에는 기본적으로 Azure CLI가 설치되어 있지 않을 수 있으므로, 아래 명령어로 설치를 진행합니다.
- 설치가 완료되면 `az --version` 명령어로 정상적으로 설치되었는지 확인하세요.
- 이후 단계에서는 az 명령어를 사용해 리소스 그룹, 컨테이너 레지스트리, AKS 클러스터 등 다양한 Azure 리소스를 생성하고 관리할 수 있습니다.
- 로컬 환경에서 실습할 경우에도 동일하게 Azure CLI를 설치해야 하며, 공식 문서의 설치 가이드를 참고할 수 있습니다.

```
curl -sL https://aka.ms/InstallAzureCLIDeb | sudo bash
```

## 5. AKS 클러스터 및 ACR 레지스트리 생성

이 단계에서는 Azure에서 컨테이너 레지스트리(ACR)와 AKS 클러스터를 생성합니다.

- AKS(Azure Kubernetes Service)는 완전관리형 Kubernetes 클러스터로, 인프라 관리 부담 없이 컨테이너 앱을 배포·운영할 수 있습니다.
- ACR(Azure Container Registry)는 Docker 이미지를 저장·관리·배포할 수 있는 Azure의 프라이빗 레지스트리입니다.
- 아래 명령어를 통해 리소스 그룹, ACR, AKS 클러스터를 순차적으로 생성합니다.
- **ACR 이름은 Azure 전체에서 유일해야 하므로, 중복되지 않도록 본인만의 고유한 이름(예: 이니셜, 날짜, 랜덤 문자열 등)을 조합해 사용하세요.**
- 클러스터 생성 시 --attach-acr 옵션을 사용하면, AKS에서 ACR 이미지를 바로 사용할 수 있도록 권한이 자동으로 연결됩니다.
- 각 명령어 실행 후 결과 메시지를 확인하고, 오류 발생 시 리소스 이름 중복, 권한, 지역(location) 설정 등을 점검하세요.
- 클러스터 인증 정보를 가져오면 kubectl로 클러스터에 바로 연결할 수 있습니다.

In [ ]:
### Azure 계정 로그인 (터미널에서 직접 실행)

아래 명령어를 터미널에서 직접 실행해 Azure 계정에 로그인하세요. (Jupyter 노트북 셀에서 실행하지 마세요)

```sh
az login
```

- 로그인 후, Azure CLI가 정상적으로 인증되었는지 `az account show`로 확인할 수 있습니다.

In [1]:
# Azure 리소스 그룹 생성 (예시)
!az group create --name myResourceGroup --location koreacentral

{
  "id": "/subscriptions/49a89096-a0ae-4e59-816b-dcb0a6fe9168/resourceGroups/myResourceGroup",
  "location": "koreacentral",
  "managedBy": null,
  "name": "myResourceGroup",
  "properties": {
    "provisioningState": "Succeeded"
  },
  "tags": null,
  "type": "Microsoft.Resources/resourceGroups"
}


In [2]:
# Azure Container Registry 생성 (예시)
# ACR 이름은 반드시 전역에서 유일해야 하므로, 중복되지 않게 본인만의 고유한 이름을 사용하세요.
!az acr create --resource-group myResourceGroup --name myacr05022025 --sku Basic

{\ Finished ..
  "adminUserEnabled": false,
  "anonymousPullEnabled": false,
  "creationDate": "2025-05-02T01:02:44.839866+00:00",
  "dataEndpointEnabled": false,
  "dataEndpointHostNames": [],
  "encryption": {
    "keyVaultProperties": null,
    "status": "disabled"
  },
  "id": "/subscriptions/49a89096-a0ae-4e59-816b-dcb0a6fe9168/resourceGroups/myResourceGroup/providers/Microsoft.ContainerRegistry/registries/myacr05022025",
  "identity": null,
  "location": "koreacentral",
  "loginServer": "myacr05022025.azurecr.io",
  "metadataSearch": "Disabled",
  "name": "myacr05022025",
  "networkRuleBypassOptions": "AzureServices",
  "networkRuleSet": null,
  "policies": {
    "azureAdAuthenticationAsArmPolicy": {
      "status": "enabled"
    },
    "exportPolicy": {
      "status": "enabled"
    },
    "quarantinePolicy": {
      "status": "disabled"
    },
    "retentionPolicy": {
      "days": 7,
      "lastUpdatedTime": "2025-05-02T01:02:54.064742+00:00",
      "status": "disabled"
    },

In [ ]:
# AKS 클러스터 생성 (예시)
# --node-vm-size 옵션을 추가하여 VM 크기를 Standard_DS4_v2로 설정합니다.
# --attach-acr 옵션을 통해 AKS 클러스터에 ACR 연결
# 아래 ACR 이름(myacr05022025)은 반드시 본인만의 고유한 이름으로 교체하세요. (중복 불가)
!az aks create --resource-group myResourceGroup --name myAKSCluster \
--node-count 1 --node-vm-size Standard_DS4_v2 \
--enable-addons monitoring --enable-managed-identity \
--generate-ssh-keys --attach-acr myacr05022025

The default value of '--node-vm-size' will be changed to 'Dynamically Selected By Azure' from 'Standard_DS2_V2 (Linux), Standard_DS2_V3 (Windows)' in next breaking change release(2.73.0) scheduled for May 2025.
 \ Finished ..{
  "aadProfile": null,
  "addonProfiles": {
    "omsagent": {
      "config": {
        "logAnalyticsWorkspaceResourceID": "/subscriptions/49a89096-a0ae-4e59-816b-dcb0a6fe9168/resourceGroups/DefaultResourceGroup-SE/providers/Microsoft.OperationalInsights/workspaces/DefaultWorkspace-49a89096-a0ae-4e59-816b-dcb0a6fe9168-SE",
        "useAADAuth": "true"
      },
      "enabled": true,
      "identity": null
    }
  },
  "agentPoolProfiles": [
    {
      "availabilityZones": null,
      "capacityReservationGroupId": null,
      "count": 1,
      "creationData": null,
      "currentOrchestratorVersion": "1.31.7",
      "eTag": null,
      "enableAutoScaling": false,
      "enableEncryptionAtHost": false,
      "enableFips": false,
      "enableNodePublicIp": false,
 

In [5]:
# AKS 클러스터 인증 정보 가져오기 및 연결
!az aks get-credentials --resource-group myResourceGroup --name myAKSCluster

Merged "myAKSCluster" as current context in /Users/junwoojeong/.kube/config


In [6]:
# 연결된 AKS 클러스터 확인 (kubectl)
!kubectl get nodes

NAME                                STATUS   ROLES    AGE     VERSION
aks-nodepool1-13278011-vmss000000   Ready    <none>   5m52s   v1.31.7


이제 위에서 생성한 ACR과 AKS를 활용해 이미지를 푸시하고, 클러스터에 배포할 수 있습니다.

## 6. Kubernetes 배포를 위한 YAML 파일 작성

이 단계에서는 AKS에 애플리케이션을 배포하기 위한 Kubernetes 배포 설정 파일(deployment.yaml)을 작성합니다.

- YAML 파일은 배포할 컨테이너 이미지, 파드 수(replicas), 리소스, 네트워크, 환경 변수 등 다양한 배포 옵션을 선언적으로 정의할 수 있습니다.
- deployment.yaml에는 앱 이름, 사용할 이미지, 파드 수, 레이블, 컨테이너 실행 명령어 등을 명시합니다.
- 실제 배포 시에는 이미지 경로(`image:`)를 본인의 ACR 주소로 교체해야 하며, 필요에 따라 환경 변수, 볼륨, 포트 등도 추가할 수 있습니다.
- YAML 파일을 활용하면 동일한 설정으로 여러 환경(개발, 테스트, 운영)에 일관되게 배포할 수 있습니다.
- 작성한 YAML 파일은 이후 kubectl apply 명령어로 클러스터에 배포할 때 사용됩니다.

In [7]:
%%writefile deployment.yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: travel-planning-agent-app
spec:
  replicas: 1
  selector:
    matchLabels:
      app: travel-planning-agent-app
  template:
    metadata:
      labels:
        app: travel-planning-agent-app
    spec:
      containers:
      - name: travel-planning-agent-app
        image: myacr05022025.azurecr.io/travel-planning-agent-app:v1  # 실제 ACR 경로(myacr05022025.azurecr.io)는 반드시 본인만의 고유한 이름으로 교체하세요. (중복 불가)

Overwriting deployment.yaml


## 7. AKS에 배포 (kubectl)

이 단계에서는 빌드한 이미지를 Azure Container Registry(ACR)에 푸시하고, 작성한 YAML 파일을 이용해 AKS 클러스터에 애플리케이션을 배포합니다.

- 먼저 이미지를 ACR에 업로드(`docker push`)하고, AKS에서 해당 이미지를 사용할 수 있도록 합니다.
- `kubectl apply -f deployment.yaml` 명령어로 YAML 파일의 설정에 따라 파드, 디플로이먼트 등 리소스가 생성됩니다.
- 배포 후 `kubectl get pods`, `kubectl get deployments` 등으로 리소스 상태를 확인할 수 있습니다.
- 배포 과정에서 이미지 경로, 네트워크 설정, 리소스 한계 등 오류가 발생할 수 있으니, 로그와 상태를 꼼꼼히 점검하세요.
- 필요에 따라 서비스(Service), 인그레스(Ingress) 등 추가 리소스를 정의해 외부 접근을 설정할 수 있습니다.

In [8]:
# 1. ACR에 로그인 (예시)
# 아래 ACR 이름(myacr05022025)은 Azure 전체에서 유일해야 하므로, 반드시 중복되지 않게 본인만의 고유한 이름을 사용하세요.
!az acr login --name myacr05022025

The output will be changed in next breaking change release(2.73.0) scheduled for May 2025. Exit code will be 1 if command fails for docker login.
Login Succeeded


In [9]:
# 2. Docker 이미지 태그 변경 (예시)
# myacr05022025.azurecr.io 경로는 본인만의 고유한 ACR 이름으로 반드시 교체하세요. (중복 불가)
!docker tag travel-planning-agent-app:v1 myacr05022025.azurecr.io/travel-planning-agent-app:v1

In [10]:
# 3. Docker 이미지를 ACR에 푸시 (예시)
# myacr05022025.azurecr.io 경로는 본인만의 고유한 ACR 이름으로 반드시 교체하세요. (중복 불가)
!docker push myacr05022025.azurecr.io/travel-planning-agent-app:v1

The push refers to repository [myacr05022025.azurecr.io/travel-planning-agent-app]

9d6e7664: Waiting 
522b6496: Waiting 
7eea74a3: Waiting 
7ff5ea4e: Waiting 
f9b9c923: Waiting 
5c05dee3: Waiting 
9887c99c: Waiting 
31d8a9a9: Waiting 
f9b9c923: Pushed   44.53MB/44.53MBWaiting Pushing  9.437MB/28.07MBv1: digest: sha256:47c1f8cfd7824c627429a310ae0055a74f6e86a1fafd84ebc3aace6293a9aa0f size: 856


In [13]:
# 4. AKS에 배포 (예시)
!kubectl apply -f deployment.yaml

deployment.apps/travel-planning-agent-app created


---

- 필요시 replicas, 리소스, 네트워크 설정 등은 YAML에서 조정하세요.

## 8. Log Analytics Workspace에서 컨테이너 로그 조회

이 단계에서는 AKS 클러스터에 배포된 컨테이너의 로그를 Log Analytics Workspace를 통해 조회하는 방법을 안내합니다.

- AKS 클러스터 생성 시 Log Analytics Workspace를 연동하면, Azure Portal에서 컨테이너의 로그와 모니터링 정보를 쉽게 확인할 수 있습니다.
- **Azure Portal에서 AKS 클러스터 > 모니터링 > 로그(Logs) 메뉴로 이동**하여 로그를 조회할 수 있습니다.
- 로그(Logs) 메뉴에서 Kusto 쿼리를 활용해 특정 컨테이너, 파드, 시간대별 로그를 조회할 수 있습니다.
- 실시간 로그 스트림이 필요하다면 Azure Portal의 "실시간 로그" 기능이나, kubectl logs 명령어를 사용할 수 있습니다.
- 로그 분석을 통해 배포 후 애플리케이션의 상태, 오류, 성능 이슈 등을 신속하게 파악할 수 있습니다.
- 필요에 따라 PodName, Namespace, LogLevel 등 다양한 조건으로 필터링하여 원하는 정보를 추출할 수 있습니다.

예시: 컨테이너 v2 로그 Kusto 쿼리 (이 쿼리는 Azure Portal의 로그(Logs) 메뉴에서 실행하세요)

```
ContainerLogV2
| where ContainerName == "travel-planning-agent-app"
| sort by TimeGenerated desc
| take 100
```

- 위 쿼리는 Python 코드 셀에서 실행할 수 없으며, Azure Portal의 AKS 클러스터 > 모니터링 > 로그(Logs) 메뉴에서 실행해야 합니다.
- 쿼리 실행 시, 컨테이너 이름 등 조건을 실제 환경에 맞게 수정할 수 있습니다.

- 필요에 따라 PodName, Namespace, LogLevel 등으로 필터링할 수 있습니다.
- 실시간 로그 스트림이 필요하다면 Azure Portal의 "실시간 로그" 기능을 활용하거나, kubectl logs 명령어도 사용할 수 있습니다.

## 참고 자료 및 공식 가이드

- [Azure Kubernetes Service(AKS) 공식 문서](https://learn.microsoft.com/azure/aks/)
- [Azure Container Registry(ACR) 공식 문서](https://learn.microsoft.com/azure/container-registry/)
- [Docker 공식 문서](https://docs.docker.com/)
- [Kubernetes 공식 문서](https://kubernetes.io/docs/)
- [Azure CLI 공식 문서](https://learn.microsoft.com/cli/azure/)

위 링크를 참고하여 각 기술의 상세 사용법과 추가 예제를 확인할 수 있습니다.

### AKS 클러스터의 Managed Identity(관리형 ID) 확인 방법

AKS 클러스터가 Managed Identity로 생성되었는지, 그리고 해당 Managed Identity의 정보를 확인하려면 아래 방법을 사용하세요.

#### 1. Azure CLI로 Managed Identity 정보 확인

```sh
az aks show --resource-group myResourceGroup --name myAKSCluster --query identity
```
- 출력 결과의 `type`이 `SystemAssigned` 또는 `UserAssigned`로 표시되면 Managed Identity가 활성화된 것입니다.
- `principalId` 값이 Managed Identity의 고유 ID입니다.

#### 2. Azure Portal에서 확인
- Azure Portal > AKS 클러스터 > [설정] > [ID] 메뉴에서 관리형 ID 정보를 확인할 수 있습니다.
- System-assigned(시스템 할당) 또는 User-assigned(사용자 할당) Managed Identity가 활성화되어 있는지 확인하세요.

#### 3. Managed Identity가 없을 경우
- AKS 클러스터를 생성할 때 `--enable-managed-identity` 옵션이 반드시 포함되어야 합니다.
- 이미 생성된 클러스터에 Managed Identity를 추가할 수는 없으므로, 필요하다면 클러스터를 새로 생성해야 합니다.

#### 4. Managed Identity의 권한 확인
- Managed Identity가 ACR 등 리소스에 접근할 수 있으려면, 해당 리소스에 적절한 역할(Role Assignment)이 부여되어야 합니다.
- Azure Portal 또는 아래 CLI 명령어로 역할 할당을 확인할 수 있습니다.

```sh
az role assignment list --assignee {{principalId}}
```

principalId는 위 1번 명령어로 확인한 값을 사용하세요.

### AKS Managed Identity에 ACR Pull 권한이 없을 때 해결 가이드

AKS 클러스터의 Managed Identity(principalId)에 ACR Pull 권한이 없어 이미지를 가져오지 못하는 경우, 아래 절차를 따라 권한을 직접 할당할 수 있습니다.

#### 1. ACR 리소스 ID 확인
```sh
az acr show --name myacr05022025 --resource-group myResourceGroup --query id --output tsv
```
- 위 명령어의 결과(예: /subscriptions/xxxx/resourceGroups/xxx/providers/Microsoft.ContainerRegistry/registries/myacr05022025)를 복사합니다.

#### 2. Managed Identity에 AcrPull 역할 할당
```sh
az role assignment create --assignee {{principalId}} --role AcrPull --scope {{ACR_RESOURCE_ID}}
```
- `{{principalId}}`는 AKS 클러스터의 Managed Identity ID입니다.
- `{{ACR_RESOURCE_ID}}`는 1번에서 복사한 값입니다.

#### 예시
```sh
az role assignment create --assignee 77603e85-fec4-40a8-940d-438e665e1f06 --role AcrPull --scope /subscriptions/xxxx/resourceGroups/myResourceGroup/providers/Microsoft.ContainerRegistry/registries/myacr05022025
```

- 이 작업을 완료하면 AKS 클러스터가 해당 ACR에서 이미지를 정상적으로 pull할 수 있습니다.
- 권한 부여 후에도 문제가 지속된다면, Azure Portal에서 역할 할당 내역을 다시 확인하세요.

### 권한 부여 후에도 문제가 계속된다면: Azure Portal에서 역할 할당 확인 방법

1. **Azure Portal 접속**
   - https://portal.azure.com 에 로그인합니다.

2. **Container Registry(ACR) 리소스 이동**
   - 왼쪽 메뉴에서 "리소스 그룹" 또는 "모든 리소스"를 클릭 후, 본인의 ACR(예: myacr05022025)을 선택합니다.

3. **[액세스 제어(IAM)] 메뉴 선택**
   - ACR 리소스 왼쪽 메뉴에서 "액세스 제어(IAM)" 또는 "Access control (IAM)"을 클릭합니다.

4. **[역할 할당] 탭에서 Managed Identity 확인**
   - "역할 할당" 또는 "Role assignments" 탭을 선택합니다.
   - 목록에서 역할이 "AcrPull"로 할당된 항목을 찾고, "이름" 또는 "Object ID"가 AKS 클러스터의 Managed Identity(principalId)와 일치하는지 확인합니다.

5. **문제가 있다면**
   - Managed Identity가 목록에 없거나, 역할이 "AcrPull"이 아니라면, CLI 또는 Portal에서 역할을 다시 할당하세요.
   - 필요시 "+ 추가" → "역할 할당 추가"를 통해 직접 할당할 수 있습니다.

6. **변경 후에도 문제가 지속된다면**
   - 역할 할당이 반영되기까지 수 분이 소요될 수 있으니 잠시 기다린 후 다시 시도하세요.
   - 여전히 문제가 해결되지 않으면, ACR와 AKS의 네트워크 설정, Managed Identity 활성화 여부, principalId 오타 등을 다시 점검하세요.

### AcrPull 권한 부여 명령어를 실행해도 할당이 안될 때 점검 및 해결 가이드

AcrPull 권한 부여 명령어를 실행해도 역할이 할당되지 않는 경우, 아래 항목을 순서대로 점검하세요.

1. **명령어 실행 계정의 권한 확인**
   - az role assignment create 명령어를 실행하는 계정이 ACR 리소스 그룹 또는 구독에 **Owner** 또는 **User Access Administrator** 권한이 있는지 확인하세요.
   - Contributor 권한만 있을 경우 역할 할당이 불가합니다.

2. **principalId와 ACR_RESOURCE_ID 값이 정확한지 확인**
   - principalId는 AKS 클러스터의 Managed Identity ID여야 하며, 오타가 없어야 합니다.
   - ACR_RESOURCE_ID는 az acr show --query id --output tsv 명령어로 얻은 전체 리소스 ID여야 합니다.

3. **이미 동일한 역할이 할당되어 있는지 확인**
   - 이미 역할이 할당되어 있으면, 추가로 할당해도 별도 메시지가 없을 수 있습니다.
   - 아래 명령어로 실제 할당 내역을 확인하세요.
     ```sh
     az role assignment list --assignee {{principalId}} --scope {{ACR_RESOURCE_ID}} --output table
     ```

4. **Azure Portal에서 직접 역할 할당 시도**
   - ACR 리소스 > [액세스 제어(IAM)] > [역할 할당 추가]에서
     - 역할: AcrPull
     - 사용자, 그룹 또는 서비스 주체: principalId(Managed Identity) 선택
     - 직접 할당이 가능하면 Portal에서 할당하세요.

5. **역할 할당이 반영되기까지 대기**
   - 역할 할당이 Azure에 반영되기까지 수 분이 소요될 수 있습니다. 2~5분 후 다시 확인하세요.

6. **Azure 정책/제한 사항 확인**
   - 조직의 Azure 정책(Policy)이나 리소스 제한으로 인해 역할 할당이 차단될 수 있습니다. 관리자에게 문의하세요.

7. **기타 문제**
   - 명령어 실행 시 에러 메시지가 있다면, 해당 메시지를 참고해 추가 조치를 취하세요.
   - 여전히 할당이 되지 않으면, Azure 지원에 문의하거나 관리자에게 권한 할당을 요청하세요.

---
위 절차를 따라 점검 후에도 문제가 해결되지 않으면, 구체적인 에러 메시지와 함께 문의해 주세요.